In [45]:
import pandas as pd
import numpy as np
import sklearn.impute
import sklearn.model_selection
import sklearn.preprocessing
from acquire import get_titanic_data, get_iris_data

## Iris Data

- Use the function defined in acquire.py to load the iris data.

In [ ]:
df = get_iris_data()

In [ ]:
df.head()

- Rename the species_name column to just species.

In [ ]:
df.rename(columns = {'species_name':'species'}, inplace = True)
df.head()

- Drop the species_id and measurement_id columns.

In [ ]:
df.drop(columns = ['species_id', 'measurement_id'], inplace = True)

In [ ]:
train, test = sklearn.model_selection.train_test_split(df, random_state=47, train_size = .8)

In [ ]:
train.head()

- Encode the species name using a sklearn label encoder. Research the inverse_transform method of the label encoder. How might this be useful?

In [ ]:
encoder = sklearn.preprocessing.OneHotEncoder()

encoder.fit(train[['species']])
                  


In [ ]:
encoder.categories_

In [2]:
encoder = sklearn.preprocessing.OneHotEncoder()
encoder.fit(train[['species']])
cols = ['embark_town_' + c for c in encoder.categories_[0]]

m = encoder.transform(train[['species']]).todense()
train = pd.concat([
    train,
    pd.DataFrame(m, columns = cols, index=train.index)
], axis = 1).drop(columns='species')

m = encoder.transform(test[['species']]).todense()
test = pd.concat([
    test,
    pd.DataFrame(m, columns = cols, index=test.index)
], axis = 1).drop(columns='species')




NameError: name 'train' is not defined

In [ ]:
train.head()

- Create a function named prep_iris that accepts the untransformed iris data, and returns the data with the transformations above applied.

In [26]:
def drop_columns(df):
    df.drop(columns = ['species_id', 'measurement_id'], inplace = True)
    return df

def rename_columns(df):
    df.rename(columns = {'species_name':'species'}, inplace = True)
    return df

def encode_species(train, test):
    encoder = sklearn.preprocessing.OneHotEncoder()
    encoder.fit(train[['species']])
    cols = ['embark_town_' + c for c in encoder.categories_[0]]

    m = encoder.transform(train[['species']]).todense()
    train = pd.concat([
        train,
        pd.DataFrame(m, columns = cols, index=train.index)
    ], axis = 1).drop(columns='species')

    m = encoder.transform(test[['species']]).todense()
    test = pd.concat([
        test,
        pd.DataFrame(m, columns = cols, index=test.index)
    ], axis = 1).drop(columns='species')    
    
    return train, test
    

def prep_iris(df):
    df = drop_columns(df)
    df = rename_columns(df)
    train, test = sklearn.model_selection.train_test_split(df, random_state=123, train_size = .8)
    train, test = encode_species(train, test)
    return train, test

In [32]:
df = get_iris_data()
df.head()

,species_id,measurement_id,sepal_length,sepal_width,petal_length,petal_width,species_name
0,1,1,5.1,3.5,1.4,0.2,setosa
1,1,2,4.9,3.0,1.4,0.2,setosa
2,1,3,4.7,3.2,1.3,0.2,setosa
3,1,4,4.6,3.1,1.5,0.2,setosa
4,1,5,5.0,3.6,1.4,0.2,setosa


In [30]:
df = drop_columns(df)

In [28]:
df = rename_columns(df)

In [31]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [33]:
train, test = prep_iris(df)

In [34]:
train.head()

,sepal_length,sepal_width,petal_length,petal_width,embark_town_setosa,embark_town_versicolor,embark_town_virginica
130,7.4,2.8,6.1,1.9,0.0,0.0,1.0
119,6.0,2.2,5.0,1.5,0.0,0.0,1.0
29,4.7,3.2,1.6,0.2,1.0,0.0,0.0
0,5.1,3.5,1.4,0.2,1.0,0.0,0.0
62,6.0,2.2,4.0,1.0,0.0,1.0,0.0


## Titanic Data

- *Use the function you defined in acquire.py to load the titanic data set.*
- *Handle the missing values in the embark_town and embarked columns.*
- *Remove the deck column.*
- *Use a label encoder to transform the embarked column.*
- Scale the age and fare columns using a min max scaler. Why might this be beneficial? When might you not want to do this?
- *Fill the missing values in age. The way you fill these values is up to you. Consider the tradeoffs of different methods.*
- Create a function named prep_titanic that accepts the untransformed titanic data, and returns the data with the transformations above applied.

In [77]:
df = get_titanic_data()
df.head(1)

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,3,male,22.0,1,0,7.25,S,Third,None,Southampton,0


In [78]:
def drop_columns_titanic(df):
    df.drop(columns = ['deck', 'embarked', 'class'], inplace = True)
    return df
df = drop_columns_titanic(df)

In [79]:
train, test = sklearn.model_selection.train_test_split(df, random_state=123, train_size=.8)

In [80]:
train.head(1)

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embark_town,alone
329,329,1,1,female,16.0,0,1,57.9792,Cherbourg,0


In [81]:
def impute_embark_town(train, test):
    train.embark_town = train.embark_town.fillna("Southampton")
    test.embark_town = test.embark_town.fillna("Southampton")
    return train, test

In [82]:
train, test = impute_embark_town(train, test)
train.head()

/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embark_town,alone
329,329,1,1,female,16.0,0,1,57.9792,Cherbourg,0
749,749,0,3,male,31.0,0,0,7.7500,Queenstown,1
203,203,0,3,male,45.5,0,0,7.2250,Cherbourg,1
421,421,0,3,male,21.0,0,0,7.7333,Queenstown,1
97,97,1,1,male,23.0,0,1,63.3583,Cherbourg,0


In [83]:
def encode_embark_town(train, test):
    encoder = sklearn.preprocessing.OneHotEncoder()
    encoder.fit(train[['embark_town']])
    cols = ['embark_town_' + c for c in encoder.categories_[0]]

    m = encoder.transform(train[['embark_town']]).todense()
    train = pd.concat([
        train,
        pd.DataFrame(m, columns=cols, index=train.index)
    ], axis=1).drop(columns='embark_town')
    
    m = encoder.transform(test[['embark_town']]).todense()
    test = pd.concat([
        test,
        pd.DataFrame(m, columns=cols, index=test.index)
    ], axis=1).drop(columns='embark_town')
    return train, test

In [84]:
train, test = encode_embark_town(train, test)

In [85]:
def impute_age(train, test):
    imputer = sklearn.impute.SimpleImputer(strategy='mean')
    imputer.fit(train[['age']])
    train.age = imputer.transform(train[['age']])
    test.age = imputer.transform(test[['age']])
    return train, test

In [122]:
train, test = impute_age(train, test)
train.age.isnull().sum()

0

In [123]:
def scale_titanic(train, test)

    scaler = sklearn.preprocessing.MinMaxScaler(copy=True, feature_range=(0,1))

    scaler = scaler.fit(train[['age', 'fare']])

    train[['age_scaled', 'fare_scaled']] = scaler.transform(train[['age', 'fare']])

    test[['age_scaled', 'fare_scaled']] = scaler.transform(test[['age', 'fare']])

train.head()

SyntaxError: invalid syntax (<ipython-input-123-5e683e8d283b>, line 1)

In [109]:
def min_max_scaler(X_train, X_test):
    """Transforms features by scaling each feature to a given range.
       Takes in X_train and X_test,
       Returns the scaler and X_train_scaled and X_test_scaled within range.
       Sensitive to outliers.
    """
    scaler = (sklearn.preprocessing.MinMaxScaler(copy=True, 
                           feature_range=(0,1))
                          .fit(X_train))
    X_train_scaled = (pd.DataFrame(scaler.transform(X_train), 
                      columns=X_train.columns, 
                      index=X_train.index))
    X_test_scaled = (pd.DataFrame(scaler.transform(X_test), 
                     columns=X_test.columns,
                     index=X_test.index))
    return scaler, X_train_scaled, X_test_scaled

In [121]:
X_train = train[['age']]
X_test = test[['age']]

X_train_scaled, X_test_scaled = min_max_scaler(X_train, X_test)

ValueError: too many values to unpack (expected 2)